In [2]:
import numpy as np
import pandas as pd

# ori_df = pd.read_csv('150Michelin_viaAPI.csv')
ori_df = pd.read_csv('01-1_Yelp_data_cleaned.csv')

###### Slicing based on cities
# NYC:
# ori_df = ori_df.iloc[:930]

# Chicago:
# ori_df = ori_df.iloc[930:1857]

# Washington:
# ori_df = ori_df.iloc[1857:2761]

# SF:
# ori_df = ori_df.iloc[2761:]

In [4]:
df = ori_df

df.head()

,name,category,city,latitude,longitude,price_level,rating,review_count,delivery,pickup,...,display_phone,distance,id,image_url,is_closed,location,phone,price,transactions,url
0,Upstate,"['Seafood', 'Wine Bars', 'Beer Bar']",New York,40.726314,-73.986490,2,4.5,1699,0,0,...,(646) 791-5400,2411.918718,ehUuSk5gPTCQmwS_ubgKRA,https://s3-media1.fl.yelpcdn.com/bphoto/RvnUVU...,False,"{'address1': '95 1st Ave', 'address2': '', 'ad...",1.646792e+10,$$,['restaurant_reservation'],https://www.yelp.com/biz/upstate-new-york-2?ad...
1,Amélie,"['French', 'Wine Bars']",New York,40.732700,-73.997660,2,4.5,2428,0,0,...,(212) 533-2962,3038.869903,ETgJqJHV7BW6pIr9Ox74sA,https://s3-media4.fl.yelpcdn.com/bphoto/cSDgVu...,False,"{'address1': '22 W 8th St', 'address2': '', 'a...",1.212533e+10,$$,['restaurant_reservation'],https://www.yelp.com/biz/am%C3%A9lie-new-york?...
2,Uglyduckling,"['American (Traditional)', 'Gastropubs']",Brooklyn,40.686023,-73.991302,2,4.0,432,1,1,...,(718) 451-3825,2174.283882,CwOAKJdX8AMz5iAoA-ZEuA,https://s3-media2.fl.yelpcdn.com/bphoto/vyH8WZ...,False,"{'address1': '166 Smith St', 'address2': '', '...",1.718451e+10,$$,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/uglyduckling-brooklyn...
3,Aunt Jake's,"['Italian', 'Wine Bars', 'Pasta Shops']",New York,40.719031,-73.997716,2,4.0,1143,1,1,...,(646) 858-0470,1538.062738,I2lgw_7DUnwD92ND4PN-Ow,https://s3-media2.fl.yelpcdn.com/bphoto/hMhn_l...,False,"{'address1': '149 Mulberry St', 'address2': No...",1.646858e+10,$$,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/aunt-jakes-new-york?a...
4,LoveMama,"['Thai', 'Malaysian', 'Vietnamese']",New York,40.730386,-73.986061,2,4.0,4136,1,1,...,(212) 254-5370,2858.772941,jjJc_CrkB2HodEinB6cWww,https://s3-media1.fl.yelpcdn.com/bphoto/bLlFKT...,False,"{'address1': '174 2nd Ave', 'address2': '', 'a...",1.212255e+10,$$,"['pickup', 'delivery', 'restaurant_reservation']",https://www.yelp.com/biz/lovemama-new-york?adj...


In [33]:
def get_reviews(restaurant):
    import requests
    from bs4 import BeautifulSoup
    import re
    import json

    name= restaurant[0]
    url = restaurant[1]
    review_list = []

    response = requests.get(url)
    if not response.status_code == 200: print('Error')
    results_page = BeautifulSoup(response.content,'lxml') 
    
    # find max page
    try: 
        page_info = results_page.find('div', class_="page-of-pages arrange_unit arrange_unit--fill").get_text().strip()
        match = re.search(r'[0-9]*$', page_info)
        max_page = match[0]
    except: max_page=5
    
    pattern = r'utm_campaign=yelp_api_v3'    
    for page in range(int(max_page)):
        if page>=5: break  # top 100 comments
            
        match= re.search(pattern,url)
        current_url = url[:match.span()[0]]+'start=%s'%(page*20)+url[match.span()[1]:]

        response = requests.get(current_url)
        if not response.status_code == 200: print('Error in Page',page)
        results_page = BeautifulSoup(response.content,'lxml')

        # get reviews
        scripts = results_page.find_all('script', {'type': 'application/ld+json'})

        for item in scripts:
            if 'review' in item.get_text():
                reviews = json.loads(item.get_text())["review"]
                for item in reviews: review_list.append(item['description'])
    
#     print(name,' ## finished')
    
    return review_list

### To avoiding re-run the following codes which cost many hours, we save the updated dataset for the other notebooks（'02_FINAL_alldata.csv')

In [9]:
# first part

city_df = df.iloc[:30,:]
city_df['reviews'] = city_df[['name','url']].apply(lambda row: get_reviews(row),axis=1)

# scraping 30 restaurants per loop
total_loop = len(df)//30
error_loop=0
loop=1

code='https://www.yelp.com/biz/upstate-new-york-2?adjust_creative=3v45QlagqMHDo2gK_6Bdaw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=3v45QlagqMHDo2gK_6Bdaw'
if city_df['reviews'].iloc[-1] == []: print(f'Click the following url for recognition, then run this cell：\n{code}')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [10]:
while (loop <= total_loop):        
    part_df=df.iloc[loop*30:(loop+1)*30,:]
    if len(part_df)==0: break
    try:
        part_df['reviews'] = part_df[['name','url']].apply(lambda row: get_reviews(row),axis=1)
        if part_df['reviews'].iloc[-1] == []: 
            print(f'Click the following url for recognition, then ONLY run this cell：\n{code}')
            break        
        city_df = pd.concat([city_df,part_df])
        
        if loop%3 == 0: print('Finish loop',loop)
        loop = loop+1
    except: 
        error_loop +=1
        if error_loop >10: print('unknown error raised'); break

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Finish loop 3
Error
Error in Page 1
Error in Page 2
Error in Page 3
Error


In [20]:
# check reviews
city_df#.tail()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,reviews
0,15-east-restaurant-new-york,"[{'alias': 'japanese', 'title': 'Japanese'}, {...","{'latitude': 40.7363799, 'longitude': -73.99213}",(212) 647-0015,3451.030051,GBWMJJ4_rDWqiXJ7AvHAOg,https://s3-media1.fl.yelpcdn.com/bphoto/gxAMg-...,False,"{'address1': '15 E 15th St', 'address2': '', '...",15 East Restaurant,1.212647e+10,$$$$,4.0,618,[],https://www.yelp.com/biz/15-east-restaurant-ne...,"[Absolutely incredible Japanese food, and one ..."
1,agern-new-york,"[{'alias': 'scandinavian', 'title': 'Scandinav...","{'latitude': 40.7525286099337, 'longitude': -7...",(646) 568-4018,5410.441889,hlJnwmsxFQ9uB3JtRwJQJg,https://s3-media1.fl.yelpcdn.com/bphoto/TiV0_a...,False,"{'address1': '89 E 42nd St', 'address2': '', '...",Agern,1.646568e+10,$$$$,4.0,167,[],https://www.yelp.com/biz/agern-new-york?adjust...,[This Scandinavian gem is a must try for any c...
2,ai-fiori-new-york,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 40.7500752, 'longitude': -73.9837...",(212) 613-8660,5041.061936,bEXTA_Gwo2oVg4gnFd9fNA,https://s3-media1.fl.yelpcdn.com/bphoto/H88_SH...,False,"{'address1': '400 5th Ave', 'address2': 'Ste 2...",Ai Fiori,1.212614e+10,$$$$,4.0,1182,[],https://www.yelp.com/biz/ai-fiori-new-york?adj...,[Ai Fiori provides one of the best dining expe...
3,aldea-new-york,"[{'alias': 'portuguese', 'title': 'Portuguese'...","{'latitude': 40.738754, 'longitude': -73.993867}",(212) 675-7223,3703.976932,A6SR9hesSJn96ps99GsZow,https://s3-media4.fl.yelpcdn.com/bphoto/bwmgiP...,False,"{'address1': '31 W 17th St', 'address2': '', '...",Aldea,1.212676e+10,$$$,4.0,595,[],https://www.yelp.com/biz/aldea-new-york?adjust...,[Fantastic Portugueses restaurant. Came with f...
4,aquavit-new-york,"[{'alias': 'scandinavian', 'title': 'Scandinav...","{'latitude': 40.76079, 'longitude': -73.97221}",(212) 307-7311,6445.566479,f7apP8A_0_KqcGQTK4V2TQ,https://s3-media2.fl.yelpcdn.com/bphoto/P2GQ7z...,False,"{'address1': '65 E 55th St', 'address2': '', '...",Aquavit,1.212308e+10,$$$$,4.0,713,[],https://www.yelp.com/biz/aquavit-new-york?adju...,[The chef's tasting menu here is such an exqui...
5,atera-new-york,"[{'alias': 'tradamerican', 'title': 'American ...","{'latitude': 40.71694, 'longitude': -74.0057}",(212) 226-1444,1589.336484,rZIMlQ76I0mXL8uNii_wVg,https://s3-media4.fl.yelpcdn.com/bphoto/QAFLZY...,False,"{'address1': '77 Worth St', 'address2': '', 'a...",Atera,1.212226e+10,$$$$,4.5,234,[],https://www.yelp.com/biz/atera-new-york?adjust...,[So amazing. We came to celebrate my husband's...
6,atomix-new-york,"[{'alias': 'korean', 'title': 'Korean'}]","{'latitude': 40.74421, 'longitude': -73.98286}",NaN,4418.154634,NsACuDqm5AVPpUocvuXONg,https://s3-media2.fl.yelpcdn.com/bphoto/VmS41x...,False,"{'address1': '104 E 30th St', 'address2': '', ...",Atomix,NaN,$$$$,4.5,35,[],https://www.yelp.com/biz/atomix-new-york?adjus...,[Dining at Atomix is truly an experience. From...
7,aureole-new-york,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.7555870079476, 'longitude': -7...",(212) 319-1660,5627.339242,UBCNhz-TSvBP3zo5PjFWrQ,https://s3-media2.fl.yelpcdn.com/bphoto/RC_ttI...,False,"{'address1': '135 W 42nd St', 'address2': '', ...",Aureole,1.212319e+10,$$$$,4.0,865,[],https://www.yelp.com/biz/aureole-new-york?adju...,[**Restaurant Week Review Winter 2019**\n\nI k...
8,uchu-new-york,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]","{'latitude': 40.72172, 'longitude': -73.99026}",(212) 203-7634,1842.002673,OINi1A9bSUUN5vi7xgi3ZQ,https://s3-media4.fl.yelpcdn.com/bphoto/KIRquq...,False,"{'address1': '217 Eldridge St', 'address2': ''...",Uchu,1.212204e+10,$$$$,4.5,60,[],https://www.yelp.com/biz/uchu-new-york?adjust_...,[BEEN HERE 4 TIMES. My absolute fav restaurant...
9,bâtard-new-york,"[{'alias': 'modern_european', 'title': 'Modern...","{'latitude': 40.7195533039813, 'longitude': -7...",(212) 219-2777,

In [19]:
city_df.to_csv('02_FINAL_alldata.csv')